In [1]:
from __future__ import annotations

import json
import cProfile
import pstats
from asyncio import sleep

In [2]:
from py_research.db import (
    DataSource,
    RecMap,
    RefMap,
    SubMap,
    SelIndex,
    DataBase
)

In [3]:
from test_db_schema import Search, Project, Task, User, Membership, Organization

In [4]:
async def dummy_loader(name: str) -> dict:
    await sleep(0.2)
    return {
        'name': name,
        'age': 20
    }

In [ ]:
data_source = DataSource(
    target=Search,
    push={
        "resultCount": Search.result_count,
        "search": Search.term,
        "results": RefMap(
            ref=Search.results,
            push={
                "project_name": Project.name,
                "project_start": Project.start,
                "project_end": Project.end,
                "project_status": Project.status,
                "tasks": RefMap(
                    ref=Project.tasks,
                    push={
                        "task_name": Task.name,
                        "task_assignees": RefMap(
                            ref=Task.assignees,
                            push=User.name,
                            match_by=User.name,
                        ),
                        "task_status": Task.status,
                    },
                ),
                "members": RefMap(
                    ref=Project.members,
                    loader=dummy_loader,
                    push={User.name, User.age},
                    rel=RecMap(
                        push={
                            Membership.role,
                        },
                    ),
                ),
            },
            pull={
                Project.org: SubMap(
                    push={
                        "organization_name": Organization.name,
                        "organization_address": Organization.address,
                        "organization_city": Organization.city,
                    },
                ),
                Project.number: SelIndex()
            },
        ),
    },
)

In [ ]:
with open("./nested_data.json") as f:
    data = json.load(f)

In [ ]:
with cProfile.Profile() as pr:
  db = DataBase()
  rec = await data_source.load([data], db=db)
  
  stats = pstats.Stats(pr)

In [ ]:
[s for s in db[Search]]

In [ ]:
stats.sort_stats("cumulative")
stats.print_stats("py_research")